In [1]:
import json
import requests
import pandas as pd
from collections import defaultdict


ok = requests.get('https://api.underdogfantasy.com/beta/v5/over_under_lines')
data = ok.json()


player_data_list2 = []
data_dict2 = defaultdict(list)
for i in range(0,len(data['over_under_lines'])):
    multiplier = (data['over_under_lines'][i]['options'][0]['payout_multiplier'])
    if multiplier == '1.0':
        bigname = (data['over_under_lines'][i]['over_under']['title'])
        stat = (data['over_under_lines'][i]['over_under']['appearance_stat']['display_stat'])
        #statid = (data['over_under_lines'][i]['over_under']['appearance_stat']['pickem_stat_id'])
        appearanceid = (data['over_under_lines'][i]['over_under']['appearance_stat']['appearance_id'])
        for entry in data['appearances']:
            if entry['id'] == appearanceid:
                playerid = entry['player_id']
        for entry in data['players']:
            if entry['id'] == playerid:
                sport = entry['sport_id']
        result = bigname.replace(stat, '').replace(' O/U', '').strip()

        line = (data['over_under_lines'][i]['stat_value'])

        player_data = {
            "Name": result,
            "League": sport,
            "Player Props": stat,
            "Value": line
        }
        player_data_list2.append(player_data)


for player_info in player_data_list2:
    league = player_info["League"]
    player_name = player_info["Name"]
    prop = player_info['Player Props']
    value = player_info['Value']
    data_dict2["League"].append(league)
    data_dict2["Player Name"].append(player_name)
    data_dict2["Market"].append(prop)
    data_dict2["Value"].append(value)

df2 = pd.DataFrame(data_dict2)
print(df2)

      League      Player Name             Market  Value
0        NBA    Kawhi Leonard          1Q Points    8.5
1        NBA    Kawhi Leonard             Points   26.5
2        NBA    Kawhi Leonard  Pts + Rebs + Asts   37.5
3        NBA    Kawhi Leonard            Assists    3.5
4        NBA    Kawhi Leonard           Rebounds    7.0
...      ...              ...                ...    ...
3898  MLBSZN    Adolis García                HRs   32.5
3899  MLBSZN  Elly De La Cruz                HRs   22.5
3900  MLBSZN      Luis Arraez               Hits  182.5
3901  MLBSZN  Elly de la Cruz       Stolen Bases   45.5
3902  MLBSZN     Esteury Ruiz       Stolen Bases   56.5

[3903 rows x 4 columns]


In [2]:
df2 = df2[df2['League'] == 'NBA']
df2['Market'].unique()[4]

'Rebounds'

In [3]:
df2['Market'].unique()

array(['1Q Points', 'Points', 'Pts + Rebs + Asts', 'Assists', 'Rebounds',
       'Rebounds + Assists', '3-Pointers Made', 'Points + Assists',
       'Points + Rebounds', '1Q Assists', 'Turnovers', 'Blocks + Steals',
       'Steals', 'Blocks', '1Q Rebounds', 'Double Doubles'], dtype=object)

In [4]:
df2['markets'] = df2['Market']


In [20]:
df2 = df2[~df2['markets'].str.contains("1Q")]
df2 = df2[~df2['markets'].str.contains("1H")]

df2['markets'] = df2['markets'].str.replace('Double Doubles','double')
df2['markets'] = df2['markets'].str.replace('3-Pointers Made','threes')
df2['markets'] = df2['markets'].str.replace('+','-', regex=False)
df2['markets'] = df2['markets'].str.replace(' ','')

df2['markets'] = df2['markets'].str.replace('pts-rebs-asts','assists_points_rebounds')
df2['markets'] = df2['markets'].str.replace('rebounds-assists','assists_rebounds')
df2['markets'] = df2['markets'].str.replace('points-assists','assists_points')
df2['markets'] = df2['markets'].str.replace('points-rebounds','points_rebounds')
df2['markets'] = df2['markets'].str.replace('blocks-steals','blocks_steals')

df2['markets'] = df2['markets'].str.lower()

array(['points', 'assists_points_rebounds', 'assists', 'rebounds',
       'assists_rebounds', 'threes', 'assists_points', 'points_rebounds',
       'turnovers', 'blocks_steals', 'steals', 'blocks', 'double'],
      dtype=object)